In [ ]:
#Compute n-grams from search list 
# Read Zotero keys from file
z = []
with open('zotero_keys.txt') as f:
    for idx, x in enumerate(f):
        y = x.split(' ')
        z.append(y[-1].strip())
        if idx == 2:
            break

# Configure Zotero with API key and group ID
group_id = z[1]
api_key = z[2][1:-1]

zot = zotero.Zotero(group_id, 'group', api_key)
cols = zot.collections()

# Get items from the top collection
itz1 = zot.everything(zot.collection_items_top(cols[-1]['key']))

# Get title, abstract, and tags from Zotero items
A = []
for idx, _ in enumerate(range(len(itz1))):
    x = itz1[idx]['data']['title']
    y = itz1[idx]['data']['abstractNote']
    z = [i['tag'] for i in itz1[idx]['data']['tags']]
    A.append([x, y, z])


# Filter and count n-grams
count4 = []
badwords = ['the', 'in', 'of', 'for', 'and', 'an', 'as', 'no', 'to', '(', 'we', 'by', 'from', 'was', 'are', 'than', 'have', 'this', 'has', 'is', 'that', 'these', 'on', 'be', 'or', 'at']
for c in A:
    d = DOI_ngram(c)
    for idx, i in enumerate(d):
        coun = 0
        for idj, j in enumerate(i[0]):
            if j in badwords:
                break
            else:
                coun += 1
        if coun == len(i[0]):
            count4.append(i)

count5 = []
for idx, i in enumerate(count4):
    coun = 0
    for idj, j in enumerate(i[0]):
        if j in badwords:
            break
        elif len(j) == 1:
            break
        else:
            coun += 1
    if coun == len(i[0]):
        count5.append(i)
count6 = dict(count5)

count7 = collections.Counter(count6)
count7.most_common()

# Search for n-grams in list B
count = 0
print(f"count of identical / position in list / total size of list")
listofngrams = []
for i in range(len(B)):
    _ngrams = searching(B[i][0], broad='yes')
    if _ngrams['meta']['count'] == 0:
        continue
    else:
        y = _ngrams['results'][0]['title']
        if y == B[i][0]:
            count += 1
            print(f"{count} / {i} / {len(B)}")
            urlll = _ngrams['results'][0]['ngrams_url']
            z = requests.get(urlll).json()
            listofngrams.append(z)

# Process n-gram results and create DataFrame
import pandas as pd

ngram = []
for i in listofngrams:
    for j in i['ngrams']:
        ngram.append((j['ngram'].lower(), j['ngram_count']))

print(len(ngram))

b = collections.Counter()
for ng, co in ngram:
    b[ng] += co

c = [[x, y] for x, y in b.items()]

# Filter out bad words and create DataFrame
badwords = ['add words to filter as necessary', ' .$', 'et al', 'et$', '3 /s', 'm 3 /s']
df = pd.DataFrame.from_records(c, columns=['ngram', 'ngram_count'])
df['ngram_tokens'] = df['ngram'].str.split().str.len()
df1 = df.dropna(axis=0)
df2 = df1.loc[(df1['ngram_tokens'] > 1) & (df1['ngram_tokens'] < 5)]
df3 = df2.sort_values(by=['ngram_count', 'ngram_tokens'], ascending=False)

for i in badwords:
    df3 = df3[~df3['ngram'].str.contains(i)]

df4 = df3.to_numpy()
df4.shape

df3.head(n=20)